In [12]:
import numpy as np
import pandas as pd
import math
import re
import sys
from collections import Counter, OrderedDict, namedtuple, defaultdict
from queue import SimpleQueue
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache

In [2]:
sys.setrecursionlimit(1500)

In [4]:
with open("17-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'...'

In [6]:
test_data_raw = r"""2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533"""



def preprocess_data (data):
    # dtype='U10'
    return np.array([list(row)  for row in data.split("\n")], dtype=int)

test_data = preprocess_data(test_data_raw)
test_data

array([[2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3],
       [3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3],
       [3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4],
       [3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2],
       [4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6],
       [1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4],
       [4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6],
       [3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3],
       [4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7],
       [4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3],
       [1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3],
       [2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5],
       [4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3]])

In [7]:
data = preprocess_data(data_raw)
data

array([[1, 2, 2, ..., 1, 1, 2],
       [3, 2, 4, ..., 5, 1, 3],
       [4, 4, 1, ..., 2, 1, 2],
       ...,
       [2, 5, 5, ..., 1, 4, 4],
       [2, 4, 4, ..., 2, 5, 1],
       [1, 4, 3, ..., 1, 3, 4]])

In [68]:
Point = namedtuple("Point", ["x", "y"])
Node = namedtuple("Node", ["point", "direction", "num_steps"])

def step_in_direction(point, direction):
        if direction == "U":
            return Point(point.x-1, point.y)
        elif direction == "D":
            return Point(point.x+1, point.y)
        elif direction == "R":
            return Point(point.x, point.y+1)
        elif direction == "L":
            return Point(point.x, point.y-1)

def is_in_field(point):
        if (point.x >= 0) and (point.x < data.shape[0]) and (point.y >= 0) and (point.y < data.shape[1]):
            return True
        else:
            return False 

def neightbors (node):
    neightbors = []
    for direction in ["U", "D", "R", "L"]:
        next_point = step_in_direction(node.point, direction)
        num_steps = node.num_steps + 1 if direction == node.direction else 1
        if (num_steps <= 3) and is_in_field(next_point):
            neightbors.append(Node(next_point, direction, num_steps))

            
    return neightbors

start = Node(Point(0,0), "U", 0)
print(start)
curr = neightbors(start)[0]
for i in range(3):
    print("Current", curr)
    print(neightbors(curr))
    curr = neightbors(curr)[-1]

Node(point=Point(x=0, y=0), direction='U', num_steps=0)
Current Node(point=Point(x=1, y=0), direction='D', num_steps=1)
[Node(point=Point(x=0, y=0), direction='U', num_steps=1), Node(point=Point(x=2, y=0), direction='D', num_steps=2), Node(point=Point(x=1, y=1), direction='R', num_steps=1)]
Current Node(point=Point(x=1, y=1), direction='R', num_steps=1)
[Node(point=Point(x=0, y=1), direction='U', num_steps=1), Node(point=Point(x=2, y=1), direction='D', num_steps=1), Node(point=Point(x=1, y=2), direction='R', num_steps=2), Node(point=Point(x=1, y=0), direction='L', num_steps=1)]
Current Node(point=Point(x=1, y=0), direction='L', num_steps=1)
[Node(point=Point(x=0, y=0), direction='U', num_steps=1), Node(point=Point(x=2, y=0), direction='D', num_steps=1), Node(point=Point(x=1, y=1), direction='R', num_steps=1)]


In [100]:



def solution (data, verbose=False):
    def queue(queue, direction):
        if len(queue) >= 3:
            queue.pop(0)
        queue.append(direction)
        return queue

    def count(queue, direction):
        counter = Counter(queue)
        return counter[direction]

    Point = namedtuple("Point", ["x", "y"])
    Node = namedtuple("Node", ["point", "direction", "num_steps"])

    def step_in_direction(point, direction):
            if direction == "U":
                return Point(point.x-1, point.y)
            elif direction == "D":
                return Point(point.x+1, point.y)
            elif direction == "R":
                return Point(point.x, point.y+1)
            elif direction == "L":
                return Point(point.x, point.y-1)

    def is_in_field(point):
            if (point.x >= 0) and (point.x < data.shape[0]) and (point.y >= 0) and (point.y < data.shape[1]):
                return True
            else:
                return False 

    def neightbors (node):
        neightbors = []
        inverse_directions = {"U": "D", "D": "U", "R":"L", "L":"R"}
        possible_directions = [i for i in ["U", "D", "R", "L"] if i != inverse_directions[node.direction]]
        for direction in possible_directions:
            next_point = step_in_direction(node.point, direction)
            num_steps = node.num_steps + 1 if direction == node.direction else 1
            if (num_steps <= 3) and is_in_field(next_point):
                neightbors.append(Node(next_point, direction, num_steps))
    
        return neightbors

    def minDistance(distances, visited):
        min = 1e7
        min_index = None

        for node, val in distances.items():
            if val < min and visited[node] == 0:
                min = val
                min_index = node
        return min_index

    start = Node(Point(0,0), "U", 0)
    end_nodes = [Node(Point(data.shape[0]-1, data.shape[1]-1), direction, max_step) for direction in ["U", "L"] for max_step in [1,2,3]]
    previous = defaultdict(list)
    previous_node = {}
    distances = {}
    distances[start] = 0
    distances_visited = {}
    visited = defaultdict(int)


    for cout in range(data.shape[0] * data.shape[1] * 4 * 3):
        if cout % 100 == 0:
            print(cout, "/", data.shape[0] * data.shape[1] * 4 * 3)

        current_node = minDistance(distances, visited)
        if not current_node:
            break
        
        if all([visited[end_node]==1 for end_node in end_nodes]):
            break
        elif cout % 100 == 0:
            print(sum([visited[end_node]==1 for end_node in end_nodes]), "/", len(end_nodes), "visited")


        visited[current_node] = 1
        distances_visited[current_node] = distances[current_node]
        current_distance = distances.pop(current_node)

        if verbose:
            print("Current point", current_node)

        for next_node in neightbors(current_node):
            
            if verbose:
                print("Next node", next_node)
            if visited[next_node] == 0:
                distance_next = data[next_node.point[0], next_node.point[1]] + current_distance
                if verbose:
                    print("New distance", distance_next)
                if (not next_node in distances) or (distance_next < distances[next_node]):
                    distances[next_node] = distance_next
                    previous[next_node] = current_node

                    if verbose:
                        print("Better! New previous node", current_node)
                else:
                    if verbose:
                        print("Not better")
            else:
                if verbose:
                    print("Already visited")


    


    end_nodes = [(node, dist) for node, dist in distances_visited.items() if (node.point.x == data.shape[0]-1) and (node.point.y == data.shape[1]-1)]
    min = 1e7
    arg_min = None
    for end_node, distance in end_nodes:
        if distance < min:
            arg_min = end_node
            min = distance


    if verbose:
        debug_board = data.copy().astype('U10')
        curr = arg_min

        while previous[curr]:
            print(curr)
            node = previous[curr]
            dir_mapping = {"U": "^", "D":"v", "R":">", "L":"<"}
            debug_board[curr.point.x, curr.point.y] = dir_mapping[curr.direction]
            curr = node
        
        print(debug_board)

    return distances_visited, min



dists, sol = solution(data)
# dists, sol = solution(test_data, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))


0 / 238572
0 / 6 visited
100 / 238572
0 / 6 visited
200 / 238572
0 / 6 visited
300 / 238572
0 / 6 visited
400 / 238572
0 / 6 visited
500 / 238572
0 / 6 visited
600 / 238572
0 / 6 visited
700 / 238572
0 / 6 visited
800 / 238572
0 / 6 visited
900 / 238572
0 / 6 visited
1000 / 238572
0 / 6 visited
1100 / 238572
0 / 6 visited
1200 / 238572
0 / 6 visited
1300 / 238572
0 / 6 visited
1400 / 238572
0 / 6 visited
1500 / 238572
0 / 6 visited
1600 / 238572
0 / 6 visited
1700 / 238572
0 / 6 visited
1800 / 238572
0 / 6 visited
1900 / 238572
0 / 6 visited
2000 / 238572
0 / 6 visited
2100 / 238572
0 / 6 visited
2200 / 238572
0 / 6 visited
2300 / 238572
0 / 6 visited
2400 / 238572
0 / 6 visited
2500 / 238572
0 / 6 visited
2600 / 238572
0 / 6 visited
2700 / 238572
0 / 6 visited
2800 / 238572
0 / 6 visited
2900 / 238572
0 / 6 visited
3000 / 238572
0 / 6 visited
3100 / 238572
0 / 6 visited
3200 / 238572
0 / 6 visited
3300 / 238572
0 / 6 visited
3400 / 238572
0 / 6 visited
3500 / 238572
0 / 6 visited
3600

# Part 2

In [102]:
test_data_2 = """111111111111
999999999991
999999999991
999999999991
999999999991"""

test_data_2 = preprocess_data(test_data_2)

In [152]:



def solution (data, verbose=False):
    def queue(queue, direction):
        if len(queue) >= 3:
            queue.pop(0)
        queue.append(direction)
        return queue

    def count(queue, direction):
        counter = Counter(queue)
        return counter[direction]

    Point = namedtuple("Point", ["x", "y"])
    Node = namedtuple("Node", ["point", "direction", "num_steps"])

    def step_in_direction(point, direction):
            if direction == "U":
                return Point(point.x-1, point.y)
            elif direction == "D":
                return Point(point.x+1, point.y)
            elif direction == "R":
                return Point(point.x, point.y+1)
            elif direction == "L":
                return Point(point.x, point.y-1)

    def is_in_field(point):
            if (point.x >= 0) and (point.x < data.shape[0]) and (point.y >= 0) and (point.y < data.shape[1]):
                return True
            else:
                return False 

    def neightbors (node):
        neightbors = []
        inverse_directions = {"U": "D", "D": "U", "R":"L", "L":"R"}
        if node.num_steps < 4:
            possible_directions = [node.direction]
        else:
            possible_directions = [i for i in ["U", "D", "R", "L"] if i != inverse_directions[node.direction]]
        for direction in possible_directions:
            next_point = step_in_direction(node.point, direction)
            num_steps = node.num_steps + 1 if direction == node.direction else 1
            if (num_steps <= 10) and is_in_field(next_point):
                neightbors.append(Node(next_point, direction, num_steps))
    
        return neightbors

    def minDistance(distances, visited):
        min = 1e7
        min_index = None

        for node, val in distances.items():
            if val < min and visited[node] == 0:
                min = val
                min_index = node
        return min_index

    start_nodes = [Node(Point(0,0), "D", 0), Node(Point(0,0), "R", 0)]
    end_nodes = [Node(Point(data.shape[0]-1, data.shape[1]-1), direction, max_step) for direction in ["U", "L"] for max_step in [1,2,3]]
    previous = defaultdict(list)
    previous_node = {}
    distances = {}
    for start_node in start_nodes:
        distances[start_node] = 0
    distances_visited = {}
    visited = defaultdict(int)


    for cout in range(data.shape[0] * data.shape[1] * 4 * 10):
        if cout % 1000 == 0:
            print(cout, "/", data.shape[0] * data.shape[1] * 4 * 10)

        current_node = minDistance(distances, visited)
        if not current_node:
            break
        
        if all([visited[end_node]==1 for end_node in end_nodes]):
            break



        visited[current_node] = 1
        distances_visited[current_node] = distances[current_node]
        current_distance = distances.pop(current_node)

        if verbose:
            print("Current point", current_node)

        for next_node in neightbors(current_node):
            
            if verbose:
                print("Next node", next_node)
            if visited[next_node] == 0:
                distance_next = data[next_node.point[0], next_node.point[1]] + current_distance
                if verbose:
                    print("New distance", distance_next)
                if (not next_node in distances) or (distance_next < distances[next_node]):
                    distances[next_node] = distance_next
                    previous[next_node] = current_node

                    if verbose:
                        print("Better! New previous node", current_node)
                else:
                    if verbose:
                        print("Not better")
            else:
                if verbose:
                    print("Already visited")


    end_nodes = [(node, dist) for node, dist in distances_visited.items() if (node.point.x == data.shape[0]-1) and (node.point.y == data.shape[1]-1) and (node.num_steps >= 4)]
    min = 1e7
    arg_min = None
    for end_node, distance in end_nodes:
        if distance < min:
            arg_min = end_node
            min = distance


    if verbose:
        debug_board = data.copy().astype('U10')
        curr = arg_min

        while previous[curr]:
            print(curr)
            node = previous[curr]
            dir_mapping = {"U": "^", "D":"v", "R":">", "L":"<"}
            debug_board[curr.point.x, curr.point.y] = dir_mapping[curr.direction]
            curr = node
        
        print(debug_board)

    return distances_visited, min



dists, sol = solution(data, verbose=False)
# dists, sol = solution(test_data, verbose=False)
# print(dists)
print(sol)
# display(sum(sol))


0 / 795240
1000 / 795240
2000 / 795240
3000 / 795240
4000 / 795240
5000 / 795240
6000 / 795240
7000 / 795240
8000 / 795240
9000 / 795240
10000 / 795240
11000 / 795240
12000 / 795240
13000 / 795240
14000 / 795240
15000 / 795240
16000 / 795240
17000 / 795240
18000 / 795240
19000 / 795240
20000 / 795240
21000 / 795240
22000 / 795240
23000 / 795240
24000 / 795240
25000 / 795240
26000 / 795240
27000 / 795240
28000 / 795240
29000 / 795240
30000 / 795240
31000 / 795240
32000 / 795240
33000 / 795240
34000 / 795240
35000 / 795240
36000 / 795240
37000 / 795240
38000 / 795240
39000 / 795240
40000 / 795240
41000 / 795240
42000 / 795240
43000 / 795240
44000 / 795240
45000 / 795240
46000 / 795240
47000 / 795240
48000 / 795240
49000 / 795240
50000 / 795240
51000 / 795240
52000 / 795240
53000 / 795240
54000 / 795240
55000 / 795240
56000 / 795240
57000 / 795240
58000 / 795240
59000 / 795240
60000 / 795240
61000 / 795240
62000 / 795240
63000 / 795240
64000 / 795240
65000 / 795240
66000 / 795240
67000 / 

In [146]:
def test(data_raw):
    data_prep = preprocess_data(data_raw)
    dists, sol = solution(data_prep, verbose=False)
    print(dists)
    print(sol)

test_data_3 = """1111"""
test("""01234567891""")
test("""012345678911""") # cannot move 11 blocks
test("""01234""")
test("""0123""") # must move at least 4 blocks

0 / 132
{Node(point=Point(x=0, y=0), direction='D', num_steps=0): 0, Node(point=Point(x=0, y=0), direction='R', num_steps=0): 0, Node(point=Point(x=0, y=1), direction='R', num_steps=1): 1, Node(point=Point(x=0, y=2), direction='R', num_steps=2): 3, Node(point=Point(x=0, y=3), direction='R', num_steps=3): 6, Node(point=Point(x=0, y=4), direction='R', num_steps=4): 10, Node(point=Point(x=0, y=5), direction='R', num_steps=5): 15, Node(point=Point(x=0, y=6), direction='R', num_steps=6): 21, Node(point=Point(x=0, y=7), direction='R', num_steps=7): 28, Node(point=Point(x=0, y=8), direction='R', num_steps=8): 36, Node(point=Point(x=0, y=9), direction='R', num_steps=9): 45, Node(point=Point(x=0, y=10), direction='R', num_steps=10): 46}
46
0 / 60
{Node(point=Point(x=0, y=0), direction='D', num_steps=0): 0, Node(point=Point(x=0, y=0), direction='R', num_steps=0): 0, Node(point=Point(x=0, y=1), direction='R', num_steps=1): 1, Node(point=Point(x=0, y=2), direction='R', num_steps=2): 3, Node(point=